## This script is used to compare the results from all 3 software.
## Written by Yusreen Shah
## Date: May 10th 2023

In [6]:
# import the libraries
import numpy as np
import pandas as pd
import re 
from Bio import SeqIO
from collections import defaultdict


## This section saves the list of all the queries in a dataframe

In [7]:
# Save the data
data = defaultdict(list)

# Read the values from the .fasta file, and save them to data
for seq_record in SeqIO.parse("CAMIH1_graph/123_CAMI.fasta", "fasta"):
    query=seq_record.id
    sequence= repr(seq_record.seq)
    length=len(seq_record)
    data['Query'].append(query)
    data['Sequence'].append(sequence)
    data['Length'].append(length)
    
# Add the data to a dataframe
df = pd.DataFrame.from_dict(data)


In [8]:
# Remove Seq(' and ') from the sequences
df['Sequence'] = df['Sequence'].str.replace('Seq(''', '')
df['Sequence'] = df['Sequence'].str.replace(')', '')
df['Sequence'] = df['Sequence'].str.strip(" \' ")
df['Sequence']=df['Sequence'].str.rstrip()

In [9]:
df=df.sort_values("Query")
df=df.reset_index(drop=True)
df

,Query,Sequence,Length
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861
2,gb|AB200915.1|-|1831-2305|ARO:3005084|dfrA31,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,474
3,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861
4,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861
...,...,...,...
693,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858
694,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858
695,gb|Y17584|+|0-858|ARO:3000899|TEM-29,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858
696,gb|Z22590|+|154-955|ARO:3001406|OXA-11,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,801


In [10]:
df.to_csv("testSequences.tsv",sep="\t")

## This section is used to compare the results from Bandage and the actual list of queries.

In [11]:
# Create a dataframe for Bandage Combined1
Bandage_Combined1= pd.read_csv('RealBandage.tsv', sep='\t')

In [12]:
#len(df.index)
Bandage_Combined1

,Query,Path,Length,Query covered by path,Query covered by hits,Mean hit identity,Total hit mismatches,Total hit gap opens,Relative length,Length discrepancy,E-value product,Sequence
0,gb|X13543.1|+|185-1046|ARO:3002533|AAC(3)-IIa,(130) 605844+ (990),861,100%,100%,96.4%,31,0,100%,0,0.0,ATGCATACGCGGAAGGCAATAACGGAGGCAATTCGAAAACTCGGAG...
1,gb|U12338|+|5055-5520|ARO:3002528|AAC(3)-Ia,(71) 402256+ (535),465,100%,100%,98.06%,9,0,100%,0,0.0,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCCAAT...
2,gb|AF162771|+|61-643|ARO:3002596|AAC(6')-IIc,(90) 567692+ (671),582,100%,100%,100%,0,0,100%,0,0.0,ATGTCCGCCAACAATGCCGCAATAGTTCTACGAGTCATGGCCGAGA...
3,gb|DQ303918|+|0-600|ARO:3002547|AAC(6')-Ib-cr,"(65) 36782492-, 3274129-, 60915537-, 2728545- ...",600,100%,100%,99.6058%,3,0,100%,0,0.0,ATGAGCAACGCAAAAACAAAGTTAGGCATCACAAAGTACAGCATCG...
4,gb|DQ303918|+|0-600|ARO:3002547|AAC(6')-Ib-cr,"(40) 24065316+, 3274129-, 60915537-, 2728545- ...",588,98%,98%,99.5635%,3,0,100%,0,0.0,AAAACAAAGTTAGGCATCACAAAGTACAGCATCGTGACCAACAGCA...
...,...,...,...,...,...,...,...,...,...,...,...,...
512,gb|Z21523|+|0-1974|ARO:3000191|tetQ,"(1106) 1240540-, 2707722+, 5350057-, 5350055+,...",1973,100%,100%,96.9484%,66,1,99.9493%,-1,0.0,GTGCGTTTCGGCAATGCATCTATTGTAGTATATTATTGCTTAATCA...
513,gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,"(39) 288518-, 23815819-, 336894- (107)",1920,100%,100%,100%,0,0,100%,0,0.0,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...
514,gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,"(39) 288518-, 6570599+ (158)",1920,100%,100%,99.6456%,7,0,100%,0,0.0,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...
515,gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,"(39) 288518-, 23815819-, 25312831- (98)",1911,99.5313%,99.5313%,99.4559%,11,0,100%,0,0.0,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...


In [13]:
Bandage_Combined_Query_Path =Bandage_Combined1[['Path','Query','Sequence']]

## Extract the start and end position from the path in Bandage

In [14]:
# Extract the start position
pattern_path_start = r'\((.*?)\)' 

In [15]:
#Extract the node position
pattern_path_node = r'\b(\d+)\s*[+-]'

In [16]:
Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Path'].apply(lambda x: re.findall(pattern_path_node, x))

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1322/75317959.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Path'].apply(lambda x: re.findall(pattern_path_node, x))


In [17]:
# Remove the braces and convert the values to integers
Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Extracted_Path'].apply(lambda x: [int(value) for value in x])

Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Extracted_Path'].astype(str)
Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Extracted_Path'].str.replace('[', '').str.replace(']', '')


/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1322/1862839196.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Extracted_Path'].apply(lambda x: [int(value) for value in x])
/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1322/1862839196.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Extracted_Path'].astype(str)
/var/folders/

In [18]:

Bandage_Combined_Query_Path.to_csv("test4.tsv",sep="\t")

In [19]:
Bandage_Combined_Query_Path['Start']=Bandage_Combined_Query_Path['Path'].str.extract(pattern_path_start, expand=False)
Bandage_Combined_Query_Path

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1322/3300280694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['Start']=Bandage_Combined_Query_Path['Path'].str.extract(pattern_path_start, expand=False)


,Path,Query,Sequence,Extracted_Path,Start
0,(130) 605844+ (990),gb|X13543.1|+|185-1046|ARO:3002533|AAC(3)-IIa,ATGCATACGCGGAAGGCAATAACGGAGGCAATTCGAAAACTCGGAG...,605844,130
1,(71) 402256+ (535),gb|U12338|+|5055-5520|ARO:3002528|AAC(3)-Ia,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCCAAT...,402256,71
2,(90) 567692+ (671),gb|AF162771|+|61-643|ARO:3002596|AAC(6')-IIc,ATGTCCGCCAACAATGCCGCAATAGTTCTACGAGTCATGGCCGAGA...,567692,90
3,"(65) 36782492-, 3274129-, 60915537-, 2728545- ...",gb|DQ303918|+|0-600|ARO:3002547|AAC(6')-Ib-cr,ATGAGCAACGCAAAAACAAAGTTAGGCATCACAAAGTACAGCATCG...,"36782492, 3274129, 60915537, 2728545",65
4,"(40) 24065316+, 3274129-, 60915537-, 2728545- ...",gb|DQ303918|+|0-600|ARO:3002547|AAC(6')-Ib-cr,AAAACAAAGTTAGGCATCACAAAGTACAGCATCGTGACCAACAGCA...,"24065316, 3274129, 60915537, 2728545",40
...,...,...,...,...,...
512,"(1106) 1240540-, 2707722+, 5350057-, 5350055+,...",gb|Z21523|+|0-1974|ARO:3000191|tetQ,GTGCGTTTCGGCAATGCATCTATTGTAGTATATTATTGCTTAATCA...,"1240540, 2707722, 5350057, 5350055, 31491343",1106
513,"(39) 288518-, 23815819-, 336894- (107)",gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...,"288518, 23815819, 336894",39
514,"(39) 288518-, 6570599+ (158)",gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...,"288518, 6570599",39
515,"(39) 288518-, 23815819-, 25312831- (98)",gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...,"288518, 23815819, 25312831",39


In [20]:
# Extract the end position
Bandage_Combined_Query_Path['End']= Bandage_Combined_Query_Path["Path"].str.split().str[-1]


/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1322/363825808.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['End']= Bandage_Combined_Query_Path["Path"].str.split().str[-1]


In [21]:
Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].str.strip('()')
Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].str.strip('+')
Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].str.strip('-')
Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].astype(int)
Bandage_Combined_Query_Path

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1322/4133168393.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].str.strip('()')
/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1322/4133168393.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].str.strip('+')
/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1322/4133168393.py:3: Settin

,Path,Query,Sequence,Extracted_Path,Start,End
0,(130) 605844+ (990),gb|X13543.1|+|185-1046|ARO:3002533|AAC(3)-IIa,ATGCATACGCGGAAGGCAATAACGGAGGCAATTCGAAAACTCGGAG...,605844,130,990
1,(71) 402256+ (535),gb|U12338|+|5055-5520|ARO:3002528|AAC(3)-Ia,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCCAAT...,402256,71,535
2,(90) 567692+ (671),gb|AF162771|+|61-643|ARO:3002596|AAC(6')-IIc,ATGTCCGCCAACAATGCCGCAATAGTTCTACGAGTCATGGCCGAGA...,567692,90,671
3,"(65) 36782492-, 3274129-, 60915537-, 2728545- ...",gb|DQ303918|+|0-600|ARO:3002547|AAC(6')-Ib-cr,ATGAGCAACGCAAAAACAAAGTTAGGCATCACAAAGTACAGCATCG...,"36782492, 3274129, 60915537, 2728545",65,580
4,"(40) 24065316+, 3274129-, 60915537-, 2728545- ...",gb|DQ303918|+|0-600|ARO:3002547|AAC(6')-Ib-cr,AAAACAAAGTTAGGCATCACAAAGTACAGCATCGTGACCAACAGCA...,"24065316, 3274129, 60915537, 2728545",40,580
...,...,...,...,...,...,...
512,"(1106) 1240540-, 2707722+, 5350057-, 5350055+,...",gb|Z21523|+|0-1974|ARO:3000191|tetQ,GTGCGTTTCGGCAATGCATCTATTGTAGTATATTATTGCTTAATCA...,"1240540, 2707722, 5350057, 5350055, 31491343",1106,194
513,"(39) 288518-, 23815819-, 336894- (107)",gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...,"288518, 23815819, 336894",39,107
514,"(39) 288518-, 6570599+ (158)",gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...,"288518, 6570599",39,158
515,"(39) 288518-, 23815819-, 25312831- (98)",gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...,"288518, 23815819, 25312831",39,98


In [22]:
# Reorder the columns in Bandage_Combined_Query_Path
Bandage_Combined_Query_Path=Bandage_Combined_Query_Path[['Query','Path','Start','Extracted_Path','End','Sequence']]

# Convert integer columns to strings
Bandage_Combined_Query_Path['Path'] = Bandage_Combined_Query_Path['Path'].astype(str)
Bandage_Combined_Query_Path['End'] = Bandage_Combined_Query_Path['End'].astype(str)
Bandage_Combined_Query_Path['Start'] = Bandage_Combined_Query_Path['Start'].astype(str)
Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Extracted_Path'].astype(str)
Bandage_Combined_Query_Path['Sequence'] = Bandage_Combined_Query_Path['Sequence'].astype(str)

# Bandage_Combined1['Query'].value_counts() 
# Merge the rows that have the same query
# group the dataframe by the 'Name' column and aggregate the data for each group
merge_Bandage_df = Bandage_Combined_Query_Path.groupby('Query').agg({'Path': ' , '.join, 'Start': ', '.join, 'End': ', '.join, 'Extracted_Path': ','.join, 'Sequence': '+ '.join}).reset_index()
merge_Bandage_df


,Query,Path,Start,End,Extracted_Path,Sequence
0,gb|AB113580|+|1329-2193|ARO:3002332|GES-3,"(93) 32422725-, 5613425+ (32) , (93) 32422725-...","93, 93","32, 32","32422725, 5613425,32422725, 32146774",ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
1,gb|AB114632|+|655-1453|ARO:3002607|aadA7,"(61) 26862905- (858) , (22) 3906318-","61, 22","858, 3906318","26862905,3906318",ATGAGTGAAAAAGTGCCCGCCGAGATTTCGGTGCAACTATCACAAG...
2,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,"(93) 32422725-, 5613425+ (32) , (93) 32422725-...","93, 93","32, 32","32422725, 5613425,32422725, 32146774",ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
3,gb|AB211124|+|0-525|ARO:3002895|SAT-1,(64) 81232484- (588),64,588,81232484,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...
4,gb|AB571865.1|-|143423-144308|ARO:3003742|mphG,"(353) 50056+, 2638631+, 6283287+, 5689755- (17...","353, 353, 171, 171, 171, 171","176, 176, 176, 176, 176, 176","50056, 2638631, 6283287, 5689755,50056, 263863...",ATGAAAAATAGAGATATTCAAAAATTAGCGGAAAGAAATGGGTTAA...
...,...,...,...,...,...,...
138,gb|X07753|+|102-882|ARO:3002652|APH(3')-VIa,"(180) 6903442+, 506744- (243) , (180) 6903442+...","180, 180","243, 241","6903442, 506744,6903442, 86408359",ATGGAATTGCCTAATATTATTCAACAATTTATTGGAAACAGCGTTT...
139,gb|X13543.1|+|185-1046|ARO:3002533|AAC(3)-IIa,(130) 605844+ (990),130,990,605844,ATGCATACGCGGAAGGCAATAACGGAGGCAATTCGAAAACTCGGAG...
140,gb|X75562|+|134-935|ARO:3001402|OXA-7,"(72) 32592258-, 758212-, 74766869- (55) , (72)...","72, 72, 72, 72, 23, 23","55, 55, 55, 55, 55, 55","32592258, 758212, 74766869,32592258, 758212, 3...",ATGAAAACATTTGCCGCATATGTAATTACTGCGTGTCTTTCAAGTA...
141,gb|Z21523|+|0-1974|ARO:3000191|tetQ,"(142) 4764923-, 2707722+, 11583809+, 43494081-...","142, 142, 1106, 1106, 142, 1106, 142, 68, 55, ...","68, 55, 68, 55, 80, 80, 43494077, 68, 55, 4349...","4764923, 2707722, 11583809, 43494081, 43494077...",GTGCGTTTCGACAATGCATCTACTGTAGTATATTATTGCTTAATCC...


In [23]:
# Add Braces to the values that now have ','
def add_braces(value):
    if ',' in value:
        return '{' + value + '}'
    else:
        return value
#merge_Bandage_df['Path'] = merge_Bandage_df['Path'].apply(add_braces)
#merge_Bandage_df['Start'] = merge_Bandage_df['Start'].apply(add_braces)
#merge_Bandage_df['End'] = merge_Bandage_df['End'].apply(add_braces)
#merge_Bandage_df['Sequence'] = merge_Bandage_df['Sequence'].apply(add_braces)

In [24]:
# Store the dataframe containing the Bandage results to a .tsv file
merge_Bandage_df.to_csv("Test1.tsv", sep="\t")

In [25]:
#Use merge operation so that we have the sequences for both softwares for successful queries
df_QueryAndResultsBandage=pd.merge(df,merge_Bandage_df, on='Query',how="outer")

## Compile the Queries and the results from Bandage in one dataframe

In [26]:
df_QueryAndResultsBandage

,Query,Sequence_x,Length,Path,Start,End,Extracted_Path,Sequence_y
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861.0,NaN,NaN,NaN,NaN,NaN
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861.0,NaN,NaN,NaN,NaN,NaN
2,gb|AB200915.1|-|1831-2305|ARO:3005084|dfrA31,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,474.0,NaN,NaN,NaN,NaN,NaN
3,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861.0,NaN,NaN,NaN,NaN,NaN
4,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
778,gb|U12338|+|5055-5520|ARO:3002528|AAC(3)-Ia,NaN,NaN,(71) 402256+ (535),71,535,402256,ATGGGCATCATTCGCACATGTAGGCTCGGCCCTGACCAAGTCCAAT...
779,gb|U63835|+|127-955|ARO:3001410|OXA-15,NaN,NaN,"(71) 32571183-, 5674312+ (158) , (71) 32571183...","71, 71","158, 158","32571183, 5674312,32571183, 24348369",ATGGCAATCCGAATCTTCGCGATACTTTTCTCCATTTTTTCTCTTG...
780,gb|X07753|+|102-882|ARO:3002652|APH(3')-VIa,NaN,NaN,"(180) 6903442+, 506744- (243) , (180) 6903442+...","180, 180","243, 241","6903442, 506744,6903442, 86408359",ATGGAATTGCCTAATATTATTCAACAATTTATTGGAAACAGCGTTT...
781,gb|X13543.1|+|185-1046|ARO:3002533|AAC(3)-IIa,NaN,NaN,(130) 605844+ (990),130,990,605844,ATGCATACGCGGAAGGCAATAACGGAGGCAATTCGAAAACTCGGAG...


In [27]:
#Print the columns from the dataframe
df_QueryAndResultsBandage.columns

Index(['Query', 'Sequence_x', 'Length', 'Path', 'Start', 'End',
       'Extracted_Path', 'Sequence_y'],
      dtype='object')

In [28]:
#Rename the columns from the merging of Bandage and Query list
df_QueryAndResultsBandage.rename(columns = {'Sequence_x':'Sequence'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'Sequence_y':'Sequence_Bandage'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'Length':'Length_Bandage'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'Path':'Path_Bandage'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'Start':'Start_Bandage'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'End':'End_Bandage'}, inplace = True)

## Save the Queries and Results from Bandage in one .tsv file

In [29]:
df_QueryAndResultsBandage.to_csv("CompiledQueriesBandage.tsv",sep="\t")

## This section is used to compare the results from SPAligner and the actual list of queries.

In [30]:
#Create a dataframe for SPAligner Combined1
SPAligner_Combined1=pd.read_csv('RealSPAligner.tsv', sep='\t',names=["Query","Start position of alignment on sequence","End position of alignment on sequence","Start position of alignment on the first edge of the path",
"End position of alignment on the last edge of the path","Path of alignment","Length of the alignment on each edge of the path","Sequence Length",
"Sequence"])
SPAligner_Combined1

,Query,Start position of alignment on sequence,End position of alignment on sequence,Start position of alignment on the first edge of the path,End position of alignment on the last edge of the path,Path of alignment,Length of the alignment on each edge of the path,Sequence Length,Sequence
0,gb|AB211124|+|0-525|ARO:3002895|SAT-1 [Escheri...,0,470,63,533,525,81232484-,470,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...
1,gb|AB116260|+|1329-2193|ARO:3002333|GES-4 [Kle...,0,809,92,901,864,32422725-,809,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
2,gb|AY494717|+|0-864|ARO:3002334|GES-5 [Escheri...,0,809,92,901,864,32422725-,809,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
3,gb|AY628199|+|203-1064|ARO:3000988|TEM-126 [Es...,0,806,223,1029,861,32381806+,806,ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...
4,gb|AY494718|+|0-864|ARO:3002335|GES-6 [Klebsie...,0,809,92,901,864,32422725-,809,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
...,...,...,...,...,...,...,...,...,...
140,gb|EU722333.1|+|1932-3852|ARO:3000196|tet32 [E...,0,1865,38,760,1920,"6390767-,31413540-","1105,760",ATGAAAATAATCAATATCGGCATTCTGGCCCATGTAGACGCAGGAA...
141,gb|M18896.2|+|206-2126|ARO:3000190|tetO [Campy...,0,1860,253,696,1920,"4147608+,3205126+","1164,696",ATGAAAATAATTAACTTAGGCATTCTGGCTCACGTTGACGCAGGAA...
142,gb|Z21523|+|0-1974|ARO:3000191|tetQ [Bacteroid...,54,1762,0,1708,1974,2707722+,1708,ATTATAAATTTAGGAATTCTTGCTCACATTGATGCAGGAAAAACTT...
143,gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW [Bu...,0,1762,38,1800,1920,288518-,1762,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...


In [31]:
#get the columns from SPAligner
SPAligner_Combined1.columns

Index(['Query', 'Start position of alignment on sequence',
       'End position of alignment on sequence',
       'Start position of alignment on the first edge of the path',
       'End position of alignment on the last edge of the path',
       'Path of alignment', 'Length of the alignment on each edge of the path',
       'Sequence Length', 'Sequence'],
      dtype='object')

In [32]:
#format the Query column from  SPAligner_Combined1 so that the Query column is the same as the one from Bandage
pattern = r'\[.*?\]'
def format_query(x):
    return re.sub(pattern,"", x).rstrip()
    


SPAligner_Combined1['Query'] = SPAligner_Combined1['Query'].map(format_query)


In [33]:
#Get the query and sequence column from SPAligner_Combined1
SPAligner_Combined_Query_Path=SPAligner_Combined1[['Query','Sequence']]


In [34]:
#Get the query and sequence column from SPAligner_Combined1
SPAligner_Combined_Query_Path=SPAligner_Combined1[['Query','Start position of alignment on the first edge of the path', 'End position of alignment on the last edge of the path', 'Sequence Length',
'Path of alignment','Sequence']]
SPAligner_Combined_Query_Path

,Query,Start position of alignment on the first edge of the path,End position of alignment on the last edge of the path,Sequence Length,Path of alignment,Sequence
0,gb|AB211124|+|0-525|ARO:3002895|SAT-1,63,533,470,525,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...
1,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,92,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
2,gb|AY494717|+|0-864|ARO:3002334|GES-5,92,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
3,gb|AY628199|+|203-1064|ARO:3000988|TEM-126,223,1029,806,861,ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...
4,gb|AY494718|+|0-864|ARO:3002335|GES-6,92,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
...,...,...,...,...,...,...
140,gb|EU722333.1|+|1932-3852|ARO:3000196|tet32,38,760,"1105,760",1920,ATGAAAATAATCAATATCGGCATTCTGGCCCATGTAGACGCAGGAA...
141,gb|M18896.2|+|206-2126|ARO:3000190|tetO,253,696,"1164,696",1920,ATGAAAATAATTAACTTAGGCATTCTGGCTCACGTTGACGCAGGAA...
142,gb|Z21523|+|0-1974|ARO:3000191|tetQ,0,1708,1708,1974,ATTATAAATTTAGGAATTCTTGCTCACATTGATGCAGGAAAAACTT...
143,gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,38,1800,1762,1920,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...


In [35]:
#Rename the columns in SPAligner
SPAligner_Combined_Query_Path.rename(columns = {'Start position of alignment on the first edge of the path':'Start_SPAligner'}, inplace = True)
SPAligner_Combined_Query_Path.rename(columns = {'End position of alignment on the last edge of the path':'End_SPAligner'}, inplace = True)
SPAligner_Combined_Query_Path.rename(columns = {'Sequence Length':"Length_SPAligner"}, inplace = True)
SPAligner_Combined_Query_Path.rename(columns = {'Path of alignment':"Path_SPAligner"}, inplace = True)
SPAligner_Combined_Query_Path.rename(columns = {'Sequence':"Sequence_SPAligner"}, inplace = True)

SPAligner_Combined_Query_Path

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1322/2596825179.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SPAligner_Combined_Query_Path.rename(columns = {'Start position of alignment on the first edge of the path':'Start_SPAligner'}, inplace = True)
/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1322/2596825179.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SPAligner_Combined_Query_Path.rename(columns = {'End position of alignment on the last edge of the path':'End_SPAligner'}, inplace = True)
/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1322/2596825179.py:4: SettingW

,Query,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner
0,gb|AB211124|+|0-525|ARO:3002895|SAT-1,63,533,470,525,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...
1,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,92,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
2,gb|AY494717|+|0-864|ARO:3002334|GES-5,92,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
3,gb|AY628199|+|203-1064|ARO:3000988|TEM-126,223,1029,806,861,ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...
4,gb|AY494718|+|0-864|ARO:3002335|GES-6,92,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
...,...,...,...,...,...,...
140,gb|EU722333.1|+|1932-3852|ARO:3000196|tet32,38,760,"1105,760",1920,ATGAAAATAATCAATATCGGCATTCTGGCCCATGTAGACGCAGGAA...
141,gb|M18896.2|+|206-2126|ARO:3000190|tetO,253,696,"1164,696",1920,ATGAAAATAATTAACTTAGGCATTCTGGCTCACGTTGACGCAGGAA...
142,gb|Z21523|+|0-1974|ARO:3000191|tetQ,0,1708,1708,1974,ATTATAAATTTAGGAATTCTTGCTCACATTGATGCAGGAAAAACTT...
143,gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,38,1800,1762,1920,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...


In [36]:
SPAligner_Combined_Query_Path

,Query,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner
0,gb|AB211124|+|0-525|ARO:3002895|SAT-1,63,533,470,525,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...
1,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,92,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
2,gb|AY494717|+|0-864|ARO:3002334|GES-5,92,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
3,gb|AY628199|+|203-1064|ARO:3000988|TEM-126,223,1029,806,861,ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...
4,gb|AY494718|+|0-864|ARO:3002335|GES-6,92,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
...,...,...,...,...,...,...
140,gb|EU722333.1|+|1932-3852|ARO:3000196|tet32,38,760,"1105,760",1920,ATGAAAATAATCAATATCGGCATTCTGGCCCATGTAGACGCAGGAA...
141,gb|M18896.2|+|206-2126|ARO:3000190|tetO,253,696,"1164,696",1920,ATGAAAATAATTAACTTAGGCATTCTGGCTCACGTTGACGCAGGAA...
142,gb|Z21523|+|0-1974|ARO:3000191|tetQ,0,1708,1708,1974,ATTATAAATTTAGGAATTCTTGCTCACATTGATGCAGGAAAAACTT...
143,gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,38,1800,1762,1920,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...


## Compile the Queries and the results from SPAligner in one dataframe

In [37]:
#Use merge operation so that we have the sequences for both softwares for successful queries
df_QueryAndResultsSPAligner=pd.merge(df,SPAligner_Combined_Query_Path, on='Query',how='right')

df_QueryAndResultsSPAligner

,Query,Sequence,Length,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner
0,gb|AB211124|+|0-525|ARO:3002895|SAT-1,NaN,NaN,63,533,470,525,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...
1,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,NaN,NaN,92,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
2,gb|AY494717|+|0-864|ARO:3002334|GES-5,NaN,NaN,92,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
3,gb|AY628199|+|203-1064|ARO:3000988|TEM-126,ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861.0,223,1029,806,861,ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...
4,gb|AY494718|+|0-864|ARO:3002335|GES-6,NaN,NaN,92,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
...,...,...,...,...,...,...,...,...
140,gb|EU722333.1|+|1932-3852|ARO:3000196|tet32,NaN,NaN,38,760,"1105,760",1920,ATGAAAATAATCAATATCGGCATTCTGGCCCATGTAGACGCAGGAA...
141,gb|M18896.2|+|206-2126|ARO:3000190|tetO,ATGAAAATAATTAACTTAGGCATTCTGGCTCACGTTGACGCAGGAA...,1920.0,253,696,"1164,696",1920,ATGAAAATAATTAACTTAGGCATTCTGGCTCACGTTGACGCAGGAA...
142,gb|Z21523|+|0-1974|ARO:3000191|tetQ,NaN,NaN,0,1708,1708,1974,ATTATAAATTTAGGAATTCTTGCTCACATTGATGCAGGAAAAACTT...
143,gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...,1920.0,38,1800,1762,1920,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...


In [38]:
df_QueryAndResultsSPAligner.to_csv("TestSPAligner.tsv",sep="\t")

In [39]:
#Rename the columns from SPAligner and Results from the Query list
df_QueryAndResultsSPAligner.rename(columns = {'Sequence_x':'Sequence'}, inplace = True)
df_QueryAndResultsSPAligner.rename(columns = {'Sequence_y':'Sequence_SPAligner'}, inplace = True)

In [40]:
df_QueryAndResultsSPAligner['Start_SPAligner']=df_QueryAndResultsSPAligner['Start_SPAligner']+1

df_QueryAndResultsSPAligner

,Query,Sequence,Length,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner
0,gb|AB211124|+|0-525|ARO:3002895|SAT-1,NaN,NaN,64,533,470,525,ATGAAGATTTCGGTGATCCCTGAGCAGGTGGCGGAAACATTGGATG...
1,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,NaN,NaN,93,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
2,gb|AY494717|+|0-864|ARO:3002334|GES-5,NaN,NaN,93,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
3,gb|AY628199|+|203-1064|ARO:3000988|TEM-126,ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861.0,224,1029,806,861,ATGAGTATTCAACATTTTCGTGTCGCCCTTATTCCCTTTTTTGCGG...
4,gb|AY494718|+|0-864|ARO:3002335|GES-6,NaN,NaN,93,901,809,864,ATGCGCTTCATTCACGCACTATTACTGGCAGGGATCGCTCACTCTG...
...,...,...,...,...,...,...,...,...
140,gb|EU722333.1|+|1932-3852|ARO:3000196|tet32,NaN,NaN,39,760,"1105,760",1920,ATGAAAATAATCAATATCGGCATTCTGGCCCATGTAGACGCAGGAA...
141,gb|M18896.2|+|206-2126|ARO:3000190|tetO,ATGAAAATAATTAACTTAGGCATTCTGGCTCACGTTGACGCAGGAA...,1920.0,254,696,"1164,696",1920,ATGAAAATAATTAACTTAGGCATTCTGGCTCACGTTGACGCAGGAA...
142,gb|Z21523|+|0-1974|ARO:3000191|tetQ,NaN,NaN,1,1708,1708,1974,ATTATAAATTTAGGAATTCTTGCTCACATTGATGCAGGAAAAACTT...
143,gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...,1920.0,39,1800,1762,1920,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...


## This section is used to compare the results from GraphAligner and the actual list of queries.

In [41]:
#Create a dataframe for GraphAligner Combined1
GraphAligner_Combined1=pd.read_csv('RealGraphAligner.tsv', sep='\t', names=["Query", "Query Length", "Query Start", 
                                        "Query End","Strand Relative Length","Path Matching","Path Length",
                                         "Start Position on Path","End Position on Path","Number of residues Matches",
                                         "Alignment Back Length","Mapping Quality","Column 1"])
GraphAligner_Combined1

,Query,Query Length,Query Start,Query End,Strand Relative Length,Path Matching,Path Length,Start Position on Path,End Position on Path,Number of residues Matches,Alignment Back Length,Mapping Quality,Column 1
0,gb|X13543.1|+|185-1046|ARO:3002533|AAC(3)-IIa ...,861,0,861,+,>605844,5771,129,990,830,861,60,NM:i:31
1,gb|U12338|+|5055-5520|ARO:3002528|AAC(3)-Ia [P...,465,0,465,+,>402256,1181,70,535,456,465,60,NM:i:9
2,gb|AF162771|+|61-643|ARO:3002596|AAC(6')-IIc [...,582,0,582,+,>567692,1259,89,671,582,582,60,NM:i:0
3,gb|DQ303918|+|0-600|ARO:3002547|AAC(6')-Ib-cr ...,600,0,600,+,<36782492<3274129<60915537<2728545,980,64,664,597,600,60,NM:i:3
4,gb|JQ808129|+|633-1188|ARO:3002546|AAC(6')-Ib ...,555,0,555,+,<2728545,896,25,580,553,555,60,NM:i:2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,gb|EU722333.1|+|1932-3852|ARO:3000196|tet32 [E...,1920,0,1920,+,<6390767<31413540,2031,38,1958,1901,1920,60,NM:i:19
145,gb|M18896.2|+|206-2126|ARO:3000190|tetO [Campy...,1920,0,1920,+,>4147608>3205126<17316950,2211,253,2173,1913,1920,60,NM:i:7
146,gb|Z21523|+|0-1974|ARO:3000191|tetQ [Bacteroid...,1974,0,1974,+,<4764923>2707722>11583809<43494081>43494077<39...,2153,141,2115,1927,1974,60,NM:i:47
147,gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW [Bu...,1920,0,1920,+,<288518<23815819<336894,2025,38,1958,1920,1920,60,NM:i:0


In [42]:
#Format the Query column from  GraphAligner_Combined1 so that the Query column is the same as the one from Bandage
pattern = r'\[.*?\]'
def format_query(x):
    return re.sub(pattern,"", x).rstrip()
    


GraphAligner_Combined1['Query'] = GraphAligner_Combined1['Query'].map(format_query)

In [43]:
# Remove < and > from the Path
GraphAligner_Combined1['Path Matching'] = GraphAligner_Combined1['Path Matching'].str.replace('>', " ")
GraphAligner_Combined1['Path Matching'] = GraphAligner_Combined1['Path Matching'].str.replace('<', " ")

## This section drops the rows with coverages < 50 from GraphAligner_Combined1

In [44]:
# Calculate the coverage for GraphAligner
GraphAligner_Combined1["Coverage"]=GraphAligner_Combined1["Alignment Back Length"]/GraphAligner_Combined1["Query Length"]*100

In [45]:
GraphAligner_Combined1

,Query,Query Length,Query Start,Query End,Strand Relative Length,Path Matching,Path Length,Start Position on Path,End Position on Path,Number of residues Matches,Alignment Back Length,Mapping Quality,Column 1,Coverage
0,gb|X13543.1|+|185-1046|ARO:3002533|AAC(3)-IIa,861,0,861,+,605844,5771,129,990,830,861,60,NM:i:31,100.0
1,gb|U12338|+|5055-5520|ARO:3002528|AAC(3)-Ia,465,0,465,+,402256,1181,70,535,456,465,60,NM:i:9,100.0
2,gb|AF162771|+|61-643|ARO:3002596|AAC(6')-IIc,582,0,582,+,567692,1259,89,671,582,582,60,NM:i:0,100.0
3,gb|DQ303918|+|0-600|ARO:3002547|AAC(6')-Ib-cr,600,0,600,+,36782492 3274129 60915537 2728545,980,64,664,597,600,60,NM:i:3,100.0
4,gb|JQ808129|+|633-1188|ARO:3002546|AAC(6')-Ib,555,0,555,+,2728545,896,25,580,553,555,60,NM:i:2,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,gb|EU722333.1|+|1932-3852|ARO:3000196|tet32,1920,0,1920,+,6390767 31413540,2031,38,1958,1901,1920,60,NM:i:19,100.0
145,gb|M18896.2|+|206-2126|ARO:3000190|tetO,1920,0,1920,+,4147608 3205126 17316950,2211,253,2173,1913,1920,60,NM:i:7,100.0
146,gb|Z21523|+|0-1974|ARO:3000191|tetQ,1974,0,1974,+,4764923 2707722 11583809 43494081 43494077 39...,2153,141,2115,1927,1974,60,NM:i:47,100.0
147,gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,1920,0,1920,+,288518 23815819 336894,2025,38,1958,1920,1920,60,NM:i:0,100.0


In [46]:
#Drop the coverages that are < 50 from GraphAligner 
GraphAligner_Combined1.drop(GraphAligner_Combined1[GraphAligner_Combined1['Coverage'] < 50].index, inplace = True)


In [47]:
GraphAligner_Combined1['Start Position on Path']=GraphAligner_Combined1['Start Position on Path']+1
GraphAligner_Combined1['End Position on Path']=GraphAligner_Combined1['End Position on Path']
GraphAligner_Combined1

,Query,Query Length,Query Start,Query End,Strand Relative Length,Path Matching,Path Length,Start Position on Path,End Position on Path,Number of residues Matches,Alignment Back Length,Mapping Quality,Column 1,Coverage
0,gb|X13543.1|+|185-1046|ARO:3002533|AAC(3)-IIa,861,0,861,+,605844,5771,130,990,830,861,60,NM:i:31,100.0
1,gb|U12338|+|5055-5520|ARO:3002528|AAC(3)-Ia,465,0,465,+,402256,1181,71,535,456,465,60,NM:i:9,100.0
2,gb|AF162771|+|61-643|ARO:3002596|AAC(6')-IIc,582,0,582,+,567692,1259,90,671,582,582,60,NM:i:0,100.0
3,gb|DQ303918|+|0-600|ARO:3002547|AAC(6')-Ib-cr,600,0,600,+,36782492 3274129 60915537 2728545,980,65,664,597,600,60,NM:i:3,100.0
4,gb|JQ808129|+|633-1188|ARO:3002546|AAC(6')-Ib,555,0,555,+,2728545,896,26,580,553,555,60,NM:i:2,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,gb|EU722333.1|+|1932-3852|ARO:3000196|tet32,1920,0,1920,+,6390767 31413540,2031,39,1958,1901,1920,60,NM:i:19,100.0
145,gb|M18896.2|+|206-2126|ARO:3000190|tetO,1920,0,1920,+,4147608 3205126 17316950,2211,254,2173,1913,1920,60,NM:i:7,100.0
146,gb|Z21523|+|0-1974|ARO:3000191|tetQ,1974,0,1974,+,4764923 2707722 11583809 43494081 43494077 39...,2153,142,2115,1927,1974,60,NM:i:47,100.0
147,gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,1920,0,1920,+,288518 23815819 336894,2025,39,1958,1920,1920,60,NM:i:0,100.0


In [48]:
#Concatenate the values from Path_Matching
GraphAligner_Combined1['Path Matching'] = GraphAligner_Combined1['Path Matching'].apply(lambda x: ', '.join(x.split()))


In [49]:
#Get the specific columns from GraphAligner
GraphAligner_Combined_Query_Path=GraphAligner_Combined1[['Query','Path Matching', 'Start Position on Path', 
'End Position on Path']]
# Convert the integer to string
GraphAligner_Combined_Query_Path['Query']=GraphAligner_Combined_Query_Path['Query'].astype(str)
GraphAligner_Combined_Query_Path['Path Matching']=GraphAligner_Combined_Query_Path['Path Matching'].astype(str)
GraphAligner_Combined_Query_Path['Start Position on Path']=GraphAligner_Combined_Query_Path['Start Position on Path'].astype(str)
GraphAligner_Combined_Query_Path['End Position on Path']=GraphAligner_Combined_Query_Path['End Position on Path'].astype(str)



/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1322/2646241093.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GraphAligner_Combined_Query_Path['Query']=GraphAligner_Combined_Query_Path['Query'].astype(str)
/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_1322/2646241093.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GraphAligner_Combined_Query_Path['Path Matching']=GraphAligner_Combined_Query_Path['Path Matching'].astype(str)
/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/i

In [50]:
#Use merge operation so that we have the sequences for both softwares for successful queries
merged_df_Graph = GraphAligner_Combined_Query_Path.groupby('Query').agg({'Path Matching':' '.join, 'Start Position on Path': ', '.join,'End Position on Path': ', '.join}).reset_index()
merged_df_Graph

,Query,Path Matching,Start Position on Path,End Position on Path
0,gb|AB113580|+|1329-2193|ARO:3002332|GES-3,32422725,93,956
1,gb|AB114632|+|655-1453|ARO:3002607|aadA7,26862905,61,858
2,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,32422725,93,956
3,gb|AB211124|+|0-525|ARO:3002895|SAT-1,81232484,64,588
4,gb|AB571865.1|-|143423-144308|ARO:3003742|mphG,"50056, 2638631, 6283287, 5689755",353,1237
...,...,...,...,...
140,gb|X07753|+|102-882|ARO:3002652|APH(3')-VIa,"6903442, 506744",180,959
141,gb|X13543.1|+|185-1046|ARO:3002533|AAC(3)-IIa,605844,130,990
142,gb|X75562|+|134-935|ARO:3001402|OXA-7,"32592258, 758212",72,872
143,gb|Z21523|+|0-1974|ARO:3000191|tetQ,"4764923, 2707722, 11583809, 43494081, 43494077...",142,2115


In [51]:
merged_df_Graph.rename(columns = {'Path Matching':'Path_GraphAligner'}, inplace = True)
merged_df_Graph.rename(columns = {'Start Position on Path':'Start_GraphAligner'}, inplace = True)
merged_df_Graph.rename(columns = {'End Position on Path':'End_GraphAligner'}, inplace = True)

In [52]:
merged_df_Graph

,Query,Path_GraphAligner,Start_GraphAligner,End_GraphAligner
0,gb|AB113580|+|1329-2193|ARO:3002332|GES-3,32422725,93,956
1,gb|AB114632|+|655-1453|ARO:3002607|aadA7,26862905,61,858
2,gb|AB116260|+|1329-2193|ARO:3002333|GES-4,32422725,93,956
3,gb|AB211124|+|0-525|ARO:3002895|SAT-1,81232484,64,588
4,gb|AB571865.1|-|143423-144308|ARO:3003742|mphG,"50056, 2638631, 6283287, 5689755",353,1237
...,...,...,...,...
140,gb|X07753|+|102-882|ARO:3002652|APH(3')-VIa,"6903442, 506744",180,959
141,gb|X13543.1|+|185-1046|ARO:3002533|AAC(3)-IIa,605844,130,990
142,gb|X75562|+|134-935|ARO:3001402|OXA-7,"32592258, 758212",72,872
143,gb|Z21523|+|0-1974|ARO:3000191|tetQ,"4764923, 2707722, 11583809, 43494081, 43494077...",142,2115


## Compile the Queries and the results from GraphAligner in one dataframe

In [53]:
#Use merge operation so that we have the sequences for both softwares for successful queries
df_QueryAndResultsGraphAligner=pd.merge(df,merged_df_Graph, on='Query',how="left")
df_QueryAndResultsGraphAligner

,Query,Sequence,Length,Path_GraphAligner,Start_GraphAligner,End_GraphAligner
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,NaN,NaN,NaN
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,NaN,NaN,NaN
2,gb|AB200915.1|-|1831-2305|ARO:3005084|dfrA31,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,474,NaN,NaN,NaN
3,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,NaN,NaN,NaN
4,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,NaN,NaN,NaN
...,...,...,...,...,...,...
693,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,NaN,NaN,NaN
694,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,NaN,NaN,NaN
695,gb|Y17584|+|0-858|ARO:3000899|TEM-29,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,NaN,NaN,NaN
696,gb|Z22590|+|154-955|ARO:3001406|OXA-11,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,801,"26012357, 758212",8,808


## Compile all the results in one master Dataframe

In [54]:
merged_df_1 = pd.merge(df_QueryAndResultsBandage, df_QueryAndResultsSPAligner, on='Query', how='inner')
merged_df_master = pd.merge(merged_df_1, df_QueryAndResultsGraphAligner, on='Query', how='inner')
#merged_df_master['Start_SPAligner']=merged_df_master['Start_SPAligner'].astype(float).astype(int)

In [55]:
#Drop some columns

merged_df_master.drop(['Sequence_y', 'Length_x','Length_y','Sequence'], axis=1)

#Rename Sequence_x
merged_df_master.rename(columns = {'Sequence_x':'Sequence_Query'}, inplace = True)



In [56]:
def compare_paths(Start_Bandage, End_Bandage,Start_GraphAligner,End_GraphAligner):
    bandage_path_start = Start_Bandage.split(',')
    bandage_path_end = End_Bandage.split(',')
    #str(Start_GraphAligner) in str(bandage_path_start) and  str(End_GraphAligner) in  str(bandage_path_end):
    
    
    if set(str(bandage_path_start)) == set([str(Start_GraphAligner)]) and set(str(bandage_path_end) )== set([str(End_GraphAligner)]):
        return 'Similar'
    elif str(Start_GraphAligner) in str(bandage_path_start) and  str(End_GraphAligner) in  str(bandage_path_end):
        return 'Full'
    else:
        return 'Different'

In [57]:
merged_df_master['BandageVSGraphAligner'] = merged_df_master.apply(lambda row: compare_paths(row['Start_Bandage'], row['End_Bandage'],row['Start_GraphAligner'],row['End_GraphAligner']), axis=1)

merged_df_master

AttributeError: 'float' object has no attribute 'split'

In [58]:
#Compare the Start, and end path from Bandage and SPAligner
merged_df_master['Start_SPAligner']=merged_df_master['Start_SPAligner'].astype(str)
merged_df_master['End_SPAligner']=merged_df_master['End_SPAligner'].astype(str)
merged_df_master['BandageVSSPAligner']=(merged_df_master['Start_Bandage'] == merged_df_master['Start_SPAligner']) & (merged_df_master['End_Bandage'] == merged_df_master['End_SPAligner'])



merged_df_master.dtypes

Query                  object
Sequence_Query         object
Length_Bandage        float64
Path_Bandage           object
Start_Bandage          object
End_Bandage            object
Extracted_Path         object
Sequence_Bandage       object
Sequence_y             object
Length_x              float64
Start_SPAligner        object
End_SPAligner          object
Length_SPAligner       object
Path_SPAligner          int64
Sequence_SPAligner     object
Sequence               object
Length_y                int64
Path_GraphAligner      object
Start_GraphAligner     object
End_GraphAligner       object
BandageVSSPAligner       bool
dtype: object

In [59]:
#Compare the Start, and end path from GraphAligner and SPAligner
merged_df_master['SPAlignerVSGraphAligner']=(merged_df_master['Start_SPAligner'] == merged_df_master['Start_GraphAligner']) & (merged_df_master['End_SPAligner'] == merged_df_master['End_GraphAligner'])

In [60]:
#Drop some columns

merged_df_master=merged_df_master.drop(['Sequence_y', 'Length_x','Length_y','Sequence'], axis=1)

## Save the Queries and  all the Results  in one .tsv file


In [61]:
merged_df_master.to_csv("QueriesAndResultsMasterList.tsv",sep='\t')

## This section saves the Nodes and Sequences from the gfa file

In [ ]:
import gfapy

# Open the GFA file
file_path = "graph1.gfa"
gfa = gfapy.Gfa.from_file(file_path)

num_segments = len(gfa.segments)

In [ ]:
data_graph = []

In [ ]:
# Store the name and sequence for each node from the graph
for segment in gfa.segments:
    data_graph.append({"Name": segment.name, "Sequence": segment.sequence})

# Convert the list to a DataFrame
df_graph = pd.DataFrame(data_graph)

# Print the DataFrame
print(df_graph)

In [ ]:
#Count the number of duplicates
#len(df_graph['Name'])-len(df_graph['Name'].drop_duplicates())

In [ ]:
num_segments

In [ ]:
#Save only one copy of each row to the dataframe
df_graph=df_graph.drop_duplicates()

In [ ]:
df_graph

In [ ]:
df_graph['Name']=df_graph['Name'].astype(str)


## Get the sequences unique to GraphAligner
## Get the dataframe containing the path from GraphAligner


In [ ]:
df_GraphAligner_Paths= pd.read_csv('GraphAlignerNotEqual.tsv', sep='\t')
#GraphAlignerNotEqual.tsv is from another script:GraphAlignerBandageScript.ipynb

In [ ]:
df_GraphAligner_Paths

In [ ]:
df_GraphAligner_Paths
df_GraphAligner_Paths.sort_index(inplace=True)
df_GraphAligner_Paths['Path Matching']=df_GraphAligner_Paths['Path Matching'].astype(str)

In [ ]:
# Function to process the Path values
def process_path(path):
    path = path.replace('<', ' ').replace('>', ' ')
    path = ' '.join(path.split('<'))
    return path

In [ ]:
df_GraphAligner_Paths['Path Matching']=df_GraphAligner_Paths['Path Matching'].apply(process_path).str.strip()


In [ ]:
df_GraphAligner_Paths
# Create a new column 'Combined_Sequence' that concatenates sequences if Path has two values


In [ ]:
df_GraphAligner_Paths[['Path1', 'Path2']] = df_GraphAligner_Paths['Path Matching'].str.split(' ', expand=True)

In [ ]:
df_GraphAligner_Paths

In [ ]:
# Get the rows where Path 2 is not None
rows_not_none = df_GraphAligner_Paths[df_GraphAligner_Paths['Path2'].notna()]

# Print the resulting dataframe
rows_not_none

In [ ]:
#Get the sequence for Path 1 and Path 2
# Merge df1 and df2 based on matching values in Column1 and Name

merged_df= pd.merge(df_GraphAligner_Paths, df_graph, left_on='Path1', right_on='Name')
merged_df = pd.merge(merged_df, df_graph, left_on='Path2', right_on='Name')

# Concatenate the sequences
merged_df

In [ ]:
extracted_sequences_Path1_df = extract_sequences_Path1(merged_df_Path1)
extracted_sequences_Path1_df
#Save the dataframe to a pdf
extracted_sequences_Path1_df.to_csv("Path1_TestSequences.tsv", sep="\t")